In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
from clin.config import PATH_REPO
import clin.modules.ebm.extract
from clin.modules import ebm
import joblib

df = joblib.load(join(PATH_REPO, 'data', 'ebm', 'ebm_interventions_cleaned.pkl'))
df = df.iloc[:100]
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]
n = len(dfe)
llm = clin.llm.get_llm('text-davinci-003')

In [ ]:
r = {}
i = 0
extractor = ebm.extract.Extractor()
n_shots = 5
interventions = extractor(i, df, nums, n_shots, llm)

In [ ]:
r["extracted_strs"] = [
    extractor(i, df, nums, n_shots, llm) for i in tqdm(range(len(df)))
]

In [ ]:
def process_ebm_lists(
    pred_list: List[str],
    gt_list: List[str],
    verbose=False,
) -> List[bool]:
    # convert to lowercase
    pred_list = [pred.strip().lower() for pred in pred_list]
    gt_list = [gt.strip().lower() for gt in gt_list]


    return pred_list, gt_list


mets_df = pd.DataFrame(
    [
        clin.eval.calculate_precision_recall_from_lists(
            *process_ebm_lists(r["extracted_strs"][i], dfe.iloc[i]["interventions"]),
            verbose=True
        )
        for i in range(len(dfe))
    ]
)
mets_dict_single = clin.eval.aggregate_precision_recall(mets_df)

In [ ]:
mets_df

In [ ]:
mets_dict_single

In [47]:
dfv = joblib.load(join(PATH_REPO, 'data', 'ebm', 'ebm_interventions_cleaned.pkl')).iloc[100:]
for i in range(len(dfv)):
    row = dfv.iloc[i]
    print(row['doc'])
    print(clin.parse.list_to_bullet_str(row['interventions']))
    print()

Safety, tolerability, and immunogenicity after 1 and 2 doses of zoster vaccine in healthy adults ≥60 years of age.

BACKGROUND Incidence and severity of herpes zoster (HZ) and postherpetic neuralgia increase with age, associated with age-related decrease in immunity to varicella-zoster virus (VZV). One dose of zoster vaccine (ZV) has demonstrated substantial protection against HZ; this study examined impact of a second dose of ZV.
METHODS Randomized, double-blind, multicenter study with 210 subjects ≥60 years old compared immunity and safety profiles after one and two doses of ZV, separated by 6 weeks, vs. placebo. Immunogenicity was evaluated using VZV interferon-gamma (IFN-γ) enzyme-linked immunospot (ELISPOT) assay and VZV glycoprotein enzyme-linked immunosorbent antibody (gpELISA) assay. Adverse experiences (AEs) were recorded on a standardized Vaccination Report Card.
RESULTS No serious vaccine-related AEs occurred. VZV IFN-γ ELISPOT geometric mean count (GMC) of spot-forming cell